In [1]:
import itertools as it

from matplotlib import pyplot as plt
from nbmetalog import nbmetalog as nbm
import numpy as np
import pandas as pd
import seaborn as sns
from teeplot import teeplot as tp


In [2]:
# prints metadata about notebook runtime
nbm.print_metadata()


context: ci
hostname: cd84e92debf2
interpreter: 3.8.12 (default, Jan 15 2022, 18:39:47)  [GCC 7.5.0]
nbcellexec: 2
nbname: postprocess-bias
nbpath: /opt/hstrat-evolutionary-inference/binder/postprocess-bias.ipynb
revision: null
session: b12332db-fec8-4965-b17c-f93d6fa01ffb
timestamp: 2023-04-30T00:59:49Z00:00


IPython==7.16.1
keyname==0.5.2
yaml==6.0
matplotlib==3.7.1
nbmetalog==0.2.6
numpy==1.21.6
pandas==1.3.5
seaborn==0.12.1
teeplot==0.5.0
re==2.2.1
ipython_genutils==0.2.0
logging==0.5.1.2
zmq==22.0.3
json==2.0.9
six==1.16.0
ipykernel==5.5.3


In [3]:
df = pd.read_csv("https://osf.io/5trmg/download")
dfdigest = np.bitwise_xor.reduce(
    pd.util.hash_pandas_object(df),
)
print("{:x}".format(dfdigest))
df


40f23b7946d11896


,a,epoch,mut_distn,num_generations,num_islands,num_niches,p_island_migration,p_niche_invasion,population_size,replicate,...,variance_pairwise_distance,variance_pairwise_distance_branch_only,mrca_depth,sackin_index,mean_evolutionary_distinctiveness,sum_evolutionary_distinctiveness,variance_evolutionary_distinctiveness,resolution,subsampling-fraction,trie-postprocess
0,collapsed-phylogeny,7,np.random.standard_normal,32768,1024,4,0.01,3.051758e-06,32768,4,...,27.305316,27.305316,1,570110,81.707306,2.677385e+06,1030.406615,NaN,NaN,NaN
1,collapsed-phylogeny,7,np.random.standard_normal,32768,1024,4,0.01,3.051758e-06,32768,41,...,26.027820,26.027820,0,548025,83.217377,2.726867e+06,1170.361609,NaN,NaN,NaN
2,collapsed-phylogeny,7,np.random.standard_normal,32768,1024,4,0.01,3.051758e-06,32768,47,...,27.705938,27.705938,0,559117,86.538544,2.835695e+06,1284.142442,NaN,NaN,NaN
3,collapsed-phylogeny,7,np.random.standard_normal,32768,1024,4,0.01,3.051758e-06,32768,15,...,28.664163,28.664163,1,583153,76.277557,2.499463e+06,1545.456043,NaN,NaN,NaN
4,collapsed-phylogeny,7,np.random.standard_normal,32768,1024,4,0.01,3.051758e-06,32768,19,...,27.416625,27.416625,0,583519,75.600433,2.477275e+06,1027.817652,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104994,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,10,...,24.562416,24.562416,1,3305,99.163609,3.242650e+04,282.320163,30.0,0.01,naive
104995,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,16,...,17.300466,17.300466,0,2923,1893.146789,6.190590e+05,250803.885040,100.0,0.01,rollback
104996,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,1,...,88.702783,88.702783,1,717008,5.744125,1.882235e+05,6.163179,100.0,1.00,rollback
104997,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,10,...,25.685331,25.685331,0,516453,28.659882,9.391270e+05,116.954707,3.0,1.00,naive


In [4]:
# phylometrics whittled down to descriptive, non-redundant set
phylometrics = [
    #     'diversity',
    "colless_like_index",
    #     'average_depth',
    #     'average_origin_time',
    #     'average_origin_time_normalized',
    #     'max_depth',
    "mean_pairwise_distance",
    #     'mean_pairwise_distance_branch_only',
    #     'num_active',
    "num_ancestors",
    #     'num_outside',
    #     'num_roots',
    #     'num_taxa',
    #     'phylogenetic_diversity',
    #     'sum_pairwise_distance',
    #     'sum_pairwise_distance_branch_only',
    #     'total_orgs',
    #     'tree_size',
    #     'variance_pairwise_distance',
    #     'variance_pairwise_distance_branch_only',
    #     'mrca_depth',
    #     'sackin_index',
    "mean_evolutionary_distinctiveness",
    #     'sum_evolutionary_distinctiveness',
    #     'variance_evolutionary_distinctiveness',
]


In [5]:
methodological_variables = [
    "resolution",
    "trie-postprocess",
]


In [6]:
evolutionary_variables = [
    "num_islands",
    "num_niches",
    "tournament_size",
]


In [7]:
sensitivity_analysis_variables = [
    "epoch",
    "mut_distn",
]


In [8]:
fixed_variables = [
    "num_generations",  # homogeneous (num generations per epoch)
    "p_island_migration",  # homogeneous
    "population_size",  # homogeneous
    "p_niche_invasion",  # only use 3.0517578125e-08
    "subsampling-fraction",  # only use 1.0
]


In [9]:
# ensure fixed variables
df = df[
    (df["p_island_migration"] == 0.01)
    & (df["p_niche_invasion"] == 3.0517578125e-08)
    & ((df["subsampling-fraction"] == 1.0) | df["subsampling-fraction"].isna())
].copy()

for fixed_variable in fixed_variables:
    assert len(df[fixed_variable].dropna().unique()) == 1, excluded_variable

df


,a,epoch,mut_distn,num_generations,num_islands,num_niches,p_island_migration,p_niche_invasion,population_size,replicate,...,variance_pairwise_distance,variance_pairwise_distance_branch_only,mrca_depth,sackin_index,mean_evolutionary_distinctiveness,sum_evolutionary_distinctiveness,variance_evolutionary_distinctiveness,resolution,subsampling-fraction,trie-postprocess
99,reconstructed-tree,7,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,46,...,24.871385,24.871385,0,537938,118.941956,3.897490e+06,4181.014677,10.0,1.0,rollback
100,reconstructed-tree,7,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,30,...,19.444399,19.444399,0,508062,123.441128,4.044919e+06,5416.806830,30.0,1.0,expected
102,reconstructed-tree,7,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,1,...,20.707268,20.707268,0,522949,122.678925,4.019943e+06,3007.903179,10.0,1.0,naive
104,reconstructed-tree,7,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,45,...,25.075196,25.075196,0,562163,124.094620,4.066333e+06,7149.002452,30.0,1.0,rollback
106,reconstructed-tree,7,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,45,...,25.087292,25.087292,0,564294,124.032479,4.064296e+06,7129.516815,100.0,1.0,expected
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104992,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,18,...,58.131399,58.131399,1,648441,5.482509,1.796509e+05,4.899477,10.0,1.0,expected
104993,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,23,...,48.388515,48.388515,1,571087,3.439714,1.127125e+05,1.327019,100.0,1.0,expected
104996,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,1,...,88.702783,88.702783,1,717008,5.744125,1.882235e+05,6.163179,100.0,1.0,rollback
104997,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,10,...,25.685331,25.685331,0,516453,28.659882,9.391270e+05,116.954707,3.0,1.0,naive


In [10]:
# exclude extraneous variable values
df = df[(df["a"] != "consolidated-phylogeny")].copy()

df


,a,epoch,mut_distn,num_generations,num_islands,num_niches,p_island_migration,p_niche_invasion,population_size,replicate,...,variance_pairwise_distance,variance_pairwise_distance_branch_only,mrca_depth,sackin_index,mean_evolutionary_distinctiveness,sum_evolutionary_distinctiveness,variance_evolutionary_distinctiveness,resolution,subsampling-fraction,trie-postprocess
99,reconstructed-tree,7,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,46,...,24.871385,24.871385,0,537938,118.941956,3.897490e+06,4181.014677,10.0,1.0,rollback
100,reconstructed-tree,7,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,30,...,19.444399,19.444399,0,508062,123.441128,4.044919e+06,5416.806830,30.0,1.0,expected
102,reconstructed-tree,7,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,1,...,20.707268,20.707268,0,522949,122.678925,4.019943e+06,3007.903179,10.0,1.0,naive
104,reconstructed-tree,7,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,45,...,25.075196,25.075196,0,562163,124.094620,4.066333e+06,7149.002452,30.0,1.0,rollback
106,reconstructed-tree,7,np.random.exponential,32768,1024,4,0.01,3.051758e-08,32768,45,...,25.087292,25.087292,0,564294,124.032479,4.064296e+06,7129.516815,100.0,1.0,expected
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104992,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,18,...,58.131399,58.131399,1,648441,5.482509,1.796509e+05,4.899477,10.0,1.0,expected
104993,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,23,...,48.388515,48.388515,1,571087,3.439714,1.127125e+05,1.327019,100.0,1.0,expected
104996,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,1,...,88.702783,88.702783,1,717008,5.744125,1.882235e+05,6.163179,100.0,1.0,rollback
104997,reconstructed-tree,2,np.random.standard_normal,32768,1,4,0.01,3.051758e-08,32768,10,...,25.685331,25.685331,0,516453,28.659882,9.391270e+05,116.954707,3.0,1.0,naive


In [11]:
df["num_niches"].unique()


array([4, 8, 1])

In [12]:
df["num_islands"].unique()


array([1024,    1])

In [13]:
df["tournament_size"].unique()


array([2, 1, 4])

In [14]:
df["a"].unique()


array(['reconstructed-tree', 'collapsed-phylogeny'], dtype=object)

In [15]:
df["regime"] = df.apply(
    lambda row: {
        (1, 1, 2): "plain",
        (1, 1, 1): "weak selection",
        (1, 1, 4): "strong selection",
        (1, 4, 2): "4 niche ecology",
        (1, 8, 2): "8 niche ecology",
        (1024, 1, 2): "spatial structure",
    }.get(
        tuple(row[["num_islands", "num_niches", "tournament_size"]]),
        np.nan,  # default
    ),
    axis="columns",
)
df = df.dropna(axis="index", subset=["regime"])
df["regime"].unique()


array(['8 niche ecology', 'spatial structure', '4 niche ecology',
       'weak selection', 'strong selection', 'plain'], dtype=object)

In [16]:
df[df["a"] == "collapsed-phylogeny"]["regime"].unique()


array(['4 niche ecology', 'weak selection', 'plain', 'strong selection',
       'spatial structure', '8 niche ecology'], dtype=object)

In [17]:
for setup, setup_df in df.groupby(sensitivity_analysis_variables):
    print(f"{setup=}")
    for regime, regime_df in setup_df.groupby(["regime"]):
        ground_truth = regime_df[(regime_df["a"] == "collapsed-phylogeny")][
            "average_origin_time_normalized"
        ].mean()
        print(f"   regime {ground_truth=}")

        for group, group_df in regime_df[
            (regime_df["a"] == "reconstructed-tree")
        ].groupby(["trie-postprocess", "resolution"]):
            err = (
                group_df["average_origin_time_normalized"].mean()
                - ground_truth
            )
            print("      ", group, len(group_df), err)


setup=(0, 'np.random.exponential')
   regime ground_truth=32765.118097476116
       ('expected', 3.0) 50 -0.025060254607524257
       ('expected', 10.0) 50 0.0009951640458893962
       ('expected', 30.0) 50 -0.00018116798673872836
       ('expected', 100.0) 50 -0.0008352655859198421
       ('naive', 3.0) 50 -0.01729209265249665
       ('naive', 10.0) 50 0.007817926580173662
       ('naive', 30.0) 50 0.006332590721285669
       ('naive', 100.0) 50 0.005748161260271445
       ('rollback', 3.0) 50 -0.021631519517541165
       ('rollback', 10.0) 50 0.003895992922480218
       ('rollback', 30.0) 50 0.0024262215119961184
       ('rollback', 100.0) 50 0.0018417920473439153
   regime ground_truth=32762.217129429
       ('expected', 3.0) 50 -0.09846468628165894
       ('expected', 10.0) 50 -0.004577625291858567
       ('expected', 30.0) 50 -0.000590280495089246
       ('expected', 100.0) 50 -0.0004321645501477178
       ('naive', 3.0) 50 -0.09000335703240125
       ('naive', 10.0) 50 0.00254921